In [ ]:
# Required libraries for the project
!pip install opencv-python

# Import necessary libraries
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import files

In [ ]:
from google.colab import files

# Upload the zip file
uploaded = files.upload()


Saving Kvasir-SEG.zip to Kvasir-SEG.zip


In [ ]:
import zipfile
import os

# Unzip the file
zip_file = "Kvasir-SEG.zip"  # Replace with the name of your zip file
destination_dir = "/content/your_folder"  # Replace with the desired destination folder

with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(destination_dir)

# List the contents of the folder to verify
os.listdir(destination_dir)


['Kvasir-SEG']

In [ ]:
# Specify paths to the images and masks
images_path = "/content/your_folder/Kvasir-SEG/images"  # Update with actual path
masks_path = "/content/your_folder/Kvasir-SEG/masks"  # Update with actual path

image_size = (256, 256)  # You can adjust the size

In [ ]:
image_size = (256, 256)  # You can adjust the size

def load_images_and_masks(images_path, masks_path):
    images = []
    masks = []

    # Sort to ensure images and masks are loaded in the correct order
    image_files = sorted(os.listdir(images_path))
    mask_files = sorted(os.listdir(masks_path))

    for img_file, mask_file in zip(image_files, mask_files):
        img_path = os.path.join(images_path, img_file)
        mask_path = os.path.join(masks_path, mask_file)

        # Load the image and mask
        image = cv2.imread(img_path)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        # Resize if necessary
        image = cv2.resize(image, image_size)
        mask = cv2.resize(mask, image_size)

        # Normalize
        image = image / 255.0  # Normalization between 0 and 1
        mask = mask / 255.0  # Masks are also normalized

        images.append(image)
        masks.append(mask)

    # Convert lists to numpy arrays
    return np.array(images), np.array(masks)

# Load images and masks
images, masks = load_images_and_masks(images_path, masks_path)

# Reshape masks to add channel dimension
masks = masks.reshape(*masks.shape, 1)  # Add a channel dimension to the masks

# Print to confirm
print(f"Loaded {len(images)} images and {len(masks)} masks")
print(f"Masks shape: {masks.shape}")  # Print the shape of masks to verify

Loaded 1000 images and 1000 masks
Masks shape: (1000, 256, 256, 1)


In [ ]:
# Imports
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models, backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

image_size = (256, 256)  # Target image size
batch_size = 32
epochs = 50


# Load images and masks
def load_images_and_masks(images_path, masks_path):
    images, masks = [], []
    image_files = sorted(os.listdir(images_path))
    mask_files = sorted(os.listdir(masks_path))
    for img_file, mask_file in zip(image_files, mask_files):
        img = cv2.imread(os.path.join(images_path, img_file))
        mask = cv2.imread(os.path.join(masks_path, mask_file), cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, image_size)
        mask = cv2.resize(mask, image_size)
        images.append(img / 255.0)  # Normalize
        masks.append(mask / 255.0)  # Normalize
    return np.array(images), np.expand_dims(np.array(masks), axis=-1)

# Load data
images, masks = load_images_and_masks(images_path, masks_path)
print(f"Loaded {len(images)} images and {len(masks)} masks.")

Loaded 1000 images and 1000 masks.


In [ ]:
# Split data
x_train, x_val, y_train, y_val = train_test_split(images, masks, test_size=0.2, random_state=42)
print(f"Training data: {x_train.shape}, {y_train.shape}")
print(f"Validation data: {x_val.shape}, {y_val.shape}")

Training data: (800, 256, 256, 3), (800, 256, 256, 1)
Validation data: (200, 256, 256, 3), (200, 256, 256, 1)


In [ ]:
# Data augmentation
datagen_args = dict(rotation_range=15,
                    width_shift_range=0.1,
                    height_shift_range=0.1,
                    shear_range=0.1,
                    zoom_range=0.1,
                    horizontal_flip=True,
                    fill_mode='nearest')

image_datagen = ImageDataGenerator(**datagen_args)
mask_datagen = ImageDataGenerator(**datagen_args)

train_image_generator = image_datagen.flow(x_train, batch_size=batch_size, seed=42)
train_mask_generator = mask_datagen.flow(y_train, batch_size=batch_size, seed=42)
train_generator = zip(train_image_generator, train_mask_generator)

In [ ]:
# U-Net model
def unet_model(input_size=(256, 256, 3)):
    inputs = layers.Input(input_size)
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)
    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)
    b = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    u1 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(b)
    u1 = layers.concatenate([u1, c2])
    c3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u1)
    u2 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c3)
    u2 = layers.concatenate([u2, c1])
    c4 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u2)
    c4 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c4)
    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(c4)
    return models.Model(inputs=[inputs], outputs=[outputs])

    # Metrics
def iou_metric(y_true, y_pred, smooth=1e-6):
    intersection = K.sum(K.flatten(y_true) * K.flatten(y_pred))
    union = K.sum(K.flatten(y_true)) + K.sum(K.flatten(y_pred)) - intersection
    return (intersection + smooth) / (union + smooth)

def dice_coefficient(y_true, y_pred, smooth=1e-6):
    intersection = K.sum(K.flatten(y_true) * K.flatten(y_pred))
    return (2 * intersection + smooth) / (K.sum(K.flatten(y_true)) + K.sum(K.flatten(y_pred)) + smooth)

# Compile model
model = unet_model()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', iou_metric, dice_coefficient])
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 256, 256, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 256, 256, 64)   │          1,792 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 256, 256, 64)   │         36,928 │ conv2d[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 128, 128, 64)   │              0 │ conv2d_1[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 128, 128, 128)  │         73,856 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 128, 128, 128)  │        147,584 │ conv2d_2[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_1           │ (None, 64, 64, 128)    │              0 │ conv2d_3[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 64, 64, 256)    │        295,168 │ max_pooling2d_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_transpose          │ (None, 128, 128, 128)  │        131,200 │ conv2d_4[0][0]         │
│ (Conv2DTranspose)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 128, 128, 256)  │              0 │ conv2d_transpose[0][0… │
│                           │                        │                │ conv2d_3[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_5 (Conv2D)         │ (None, 128, 128, 128)  │        295,040 │ concatenate[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_transpose_1        │ (None, 256, 256, 64)   │         32,832 │ conv2d_5[0][0]         │
│ (Conv2DTranspose)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_1             │ (None, 256, 256, 128)  │              0 │ conv2d_transpose_1[0]… │
│ (Concatenate)             │                        │                │ conv2d_1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_6 (Conv2D)         │ (None, 256, 256, 64)   │         73,792 │ concatenate_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_7 (Conv2D)         │ (None, 256, 256, 64)   │         36,928 │ conv2d_6[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_8 (Conv2D)         │ (None, 256, 256, 1)    │             65 │ conv2d_7[0][0]         │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 1,125,185 (4.29 MB)

 Trainable params: 1,125,185 (4.29 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
def custom_generator(image_generator, mask_generator):
    """Yields batches of images and masks."""
    while True:
        # Get the next batch of images and masks from the generators
        image_batch = next(image_generator)
        mask_batch = next(mask_generator)
        # Yield the image batch and mask batch as a tuple
        yield image_batch, mask_batch


# Train model
history = model.fit(custom_generator(train_image_generator, train_mask_generator),  # Use the custom generator
                    steps_per_epoch=len(x_train) // batch_size,
                    epochs=epochs,
                    validation_data=(x_val, y_val))

Epoch 1/50


UnknownError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 685, in <lambda>

  File "/usr/local/lib/python3.10/di

In [ ]:
# Train model
history = model.fit(train_generator,
                    steps_per_epoch=len(x_train) // batch_size,
                    epochs=epochs,
                    validation_data=(x_val, y_val))

ValueError: Unrecognized data type: x=<zip object at 0x7893f52a4a40> (of type <class 'zip'>)